In [1]:
# Connect Driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bước 2 - Khai báo thư viện

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

Bước 3 - Khai báo features

In [3]:
CSV_HEADER=['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label', 'attack_cat'] # bo Src_IP, Dst_IP, Timestamp, {'Flow ID', 'Src Port', 'Dst Port', 'Protocol'}
CSV_HEADER_WITHOUT_LABEL = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

# Bảng mã hóa
encoded_to_original = {
    0: 'Benign',
    1: 'Analysis',
    2: 'Backdoor',
    3: 'DoS',
    4: 'Exploits',
    5: 'Fuzzers',
    6: 'Generic',
    7: 'Reconnaissance',
    8: 'Shellcode',
    9: 'Worms'
}
# decoded_labels = [encoded_to_original[label] for label in y_pred]
# decoded_labels

Bước 4 - Đọc và xử lý dữ liệu đầu vào

In [4]:
#df = pd.read_csv("/content/drive/MyDrive/BT3/CIC_UNSW-LABEL_Attack_cat.csv")
df = pd.read_csv('/content/drive/MyDrive/AnToanHeThongThongTin-CH2024/CIC_UNSW-LABEL_Attack_cat.csv')
df = df.drop(['attack_cat'], axis=1)

In [5]:
scaler = StandardScaler()
scaler.fit(df[CSV_HEADER_WITHOUT_LABEL])
df[CSV_HEADER_WITHOUT_LABEL] = scaler.transform(df[CSV_HEADER_WITHOUT_LABEL])

Chia tập train và tập test

In [6]:

# Chia dữ liệu thành train/test
train_data, test_data = train_test_split(df, test_size=0.2)
train_data_1, train_data_2 = train_test_split(train_data, test_size=0.5)
test_data.to_csv("test_data.csv", index=False, header=True)

In [7]:
# Xác định đặc trưng (X) và nhãn (y)
X_train_1 = train_data_1[CSV_HEADER_WITHOUT_LABEL]
y_train_1 = train_data_1['Label']

X_train_2 = train_data_2[CSV_HEADER_WITHOUT_LABEL]
y_train_2 = train_data_2['Label']

X_test = test_data[CSV_HEADER_WITHOUT_LABEL]
y_test = test_data['Label']

Bước 5 - Thực hiện train mô hình

In [8]:
# Huấn luyện mô hình Logistic Regression cho client 1 và client 2
model_1 = LogisticRegression(max_iter=1000)
model_1.fit(X_train_1, y_train_1)

model_2 = LogisticRegression(max_iter=1000)
model_2.fit(X_train_2, y_train_2)


LogisticRegression(max_iter=1000)

Hàm lấy mean-based của mô hình

In [9]:
# Lấy các trọng số từ mô hình của mỗi client
coef_1 = model_1.coef_
intercept_1 = model_1.intercept_

coef_2 = model_2.coef_
intercept_2 = model_2.intercept_
# Find the missing class
missing_class = set(model_2.classes_) - set(model_1.classes_)

# Add missing class to model_1 coef_ with 0 weights
for c in missing_class:
    coef_1 = np.vstack([coef_1, np.zeros((1, coef_1.shape[1]))])
    intercept_1 = np.append(intercept_1, 0)

# Now you should be able to average the coefficients
# avg_coef = (coef_1 + coef_2) / 2
# avg_intercept = (intercept_1 + intercept_2) / 2

In [10]:
# Tính kích thước của dữ liệu mỗi client
size_train_1 = len(X_train_1)
size_train_2 = len(X_train_2)

# Tính trọng số của mỗi client dựa trên kích thước dữ liệu
total_size = size_train_1 + size_train_2
weight_1 = size_train_1 / total_size
weight_2 = size_train_2 / total_size

# # Lấy các trọng số từ mô hình của mỗi client
# coef_1 = model_1.coef_
# intercept_1 = model_1.intercept_

# coef_2 = model_2.coef_
# intercept_2 = model_2.intercept_

# Tổng hợp trọng số theo công thức Weighted Federated Averaging
avg_coef = weight_1 * coef_1 + weight_2 * coef_2
avg_intercept = weight_1 * intercept_1 + weight_2 * intercept_2

# Tạo mô hình mới với trọng số tổng hợp
model_avg = LogisticRegression()
model_avg.coef_ = avg_coef
model_avg.intercept_ = avg_intercept
# Added line below to fix the error
model_avg.classes_ = model_1.classes_ # or model_2.classes_, they should be the same


Hàm tính mean-based từ 2 model đã train với 2 dữ liệu train khác nhau.

In [11]:
import numpy as np
from sklearn.linear_model import LogisticRegression

# Lấy danh sách trọng số từ các client
coefs = [model_1.coef_, model_2.coef_]  # Thay đổi theo số lượng client
intercepts = [model_1.intercept_, model_2.intercept_]

# Ensure coefs have the same shape before calculating median
# Find the union of classes from both models
all_classes = sorted(list(set(model_1.classes_) | set(model_2.classes_)))

# Reshape coefs to include all classes, filling missing ones with 0
coefs_reshaped = []
for model_coef, model_classes in zip(coefs, [model_1.classes_, model_2.classes_]):
    coef_reshaped = np.zeros((len(all_classes), model_coef.shape[1]))
    for i, cls in enumerate(model_classes):
        coef_reshaped[all_classes.index(cls)] = model_coef[i]
    coefs_reshaped.append(coef_reshaped)

# Reshape intercepts to include all classes, filling missing ones with 0
intercepts_reshaped = []
for model_intercept, model_classes in zip(intercepts, [model_1.classes_, model_2.classes_]):
    intercept_reshaped = np.zeros(len(all_classes))
    for i, cls in enumerate(model_classes):
        intercept_reshaped[all_classes.index(cls)] = model_intercept[i]
    intercepts_reshaped.append(intercept_reshaped)

# Now calculate the median with reshaped coefficients
median_coef = np.median(coefs_reshaped, axis=0)
median_intercept = np.median(intercepts_reshaped, axis=0)

# Tạo mô hình mới với trọng số tổng hợp
model_avg = LogisticRegression()
model_avg.coef_ = median_coef
model_avg.intercept_ = median_intercept
model_avg.classes_ = np.array(all_classes)  # Set classes to the union of classes

Thực hiện Test

In [12]:
# Dự đoán và đánh giá mô hình trên tập test
y_pred = model_avg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Mean-based model: {accuracy}")

Accuracy of Mean-based model: 0.8974024089391961


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Dự đoán kết quả từ mô hình tổng hợp trên tập kiểm tra
y_pred = model_avg.predict(X_test)

# In ra một số thông tin cơ bản
print(f"Predictions: {y_pred[:10]}")  # In 10 kết quả đầu tiên


Predictions: [0 0 0 0 0 0 0 0 0 5]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


DEMO 1 - Thực hiện test với mô hình global vừa tổng hợp được.

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Dự đoán với mô hình tổng hợp
y_pred = model_avg.predict(X_test)

# Đánh giá hiệu suất mô hình
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
f1 = f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))

# In ra các chỉ số đánh giá
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

# Tính Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# In báo cáo phân loại (classification report)
report = classification_report(y_test, y_pred, target_names=encoded_to_original.values())
print("\nClassification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Accuracy: 89.74%
Precision: 0.92
Recall: 0.90
F1-score: 0.90

Confusion Matrix:
[[69686     0     0     0   243  1583    12   156     0     0]
 [    0     0     0     0    67     0     0     0     0     0]
 [    2     0     6     0    28    43     0     4     0     0]
 [   11     1     1    15   465   322     2    62     0     0]
 [   64     1     0    12  3912  2119    77    87     0     0]
 [   17     0     0     0   466  4605    14   744     0     0]
 [   15     0     0     3   341   346   201    19     0     0]
 [   94     0     0     0    30  1258     0  1967     0     0]
 [    0     0     0     0    12   298     0   112     0     0]
 [    1     0     0     0    20    38     0     1     0     0]]

Classification Report:
                 precision    recall  f1-score   support

        Benign       1.00      0.97      0.98     71680
      Analysis       0.00      0.00      0.00        67
      Backdoor       0.86      0.07      0.13        83
           DoS       0.50      0.02    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
